In [1]:
import pymongo
import pandas
from pprint import pprint

# Connect
client = pymongo.MongoClient(host="mongo", port=27017, username="imdb", password="imdb_admin")

In [2]:
df1 = pandas.read_csv("IMDB-movies.csv")
df2 = pandas.read_csv("IMDB-directors.csv",)
df3 = pandas.read_csv("IMDB-movies_directors.csv")
df4 = pandas.read_csv("IMDB-movies_genres.csv")
df5 = df4.merge(df1).merge(df3).merge(df2)

df4 = df4.merge(df1)
df3 = df3.merge(df1).merge(df2)

db = client['imdb_database']
collection1 = db['imdb_movies']
collection2 = db['imdb_directors']
collection3 = db['imdb_movies_directors']
collection4 = db['imdb_movies_genres']
collection5 = db['imdb_movies_directors_genres']

df1.reset_index(inplace=True)
data_dict = df1.to_dict("records")# Insert collection
collection1.insert_many(data_dict)

df2.reset_index(inplace=True)
data_dict = df2.to_dict("records")# Insert collection
collection2.insert_many(data_dict)

df3.reset_index(inplace=True)
data_dict = df3.to_dict("records")# Insert collection
collection3.insert_many(data_dict)

df4.reset_index(inplace=True)
data_dict = df4.to_dict("records")# Insert collection
collection4.insert_many(data_dict)

df5.reset_index(inplace=True)
data_dict = df5.to_dict("records")# Insert collection
collection5.insert_many(data_dict)

In [3]:
df1.columns,df2.columns,df3.columns,df4.columns#,df5.columns

(Index(['index', 'movie_id', 'name', 'year', 'rank'], dtype='object'),
 Index(['index', 'director_id', 'first_name', 'last_name'], dtype='object'),
 Index(['index', 'director_id', 'movie_id', 'name', 'year', 'rank',
        'first_name', 'last_name'],
       dtype='object'),
 Index(['index', 'movie_id', 'genre', 'name', 'year', 'rank'], dtype='object'))

# Introduction to MongoDB (MongoDB Query Language)

##### Version 0.1

***

By Scott Coughlin (Northwestern IT Research Computing Services)  
20 July 2022

In our introduction to MongoDB we will start with queries of existing tables.

## Problem 1) IMDb Data

Throughout the session we will use information from the [Internet Movie Database (IMDb)](https://www.imdb.com/) to illustrate various principles regarding databases.

A quick note on the provenance of this data. The files we have used to populate this data set are from [this website](https://relational.fit.cvut.cz/dataset/IMDb) and it may not be a list of every single movie on IMDb (there are no movies after 2004).

For this exercise there are 5 collections, 
```
collection1 = db['imdb_movies']
collection2 = db['imdb_directors']
collection3 = db['imdb_movies_directors']
collection4 = db['imdb_movies_genres']
collection5 = db['imdb_movies_directors_genres']
```
To make things simple, I have already performed the necessary steps to "join" the information from imdb_movies and imdb_directories together to make a bigger collection "imdb_movies_directors" and so on

#### HELPFUL TIP: Convery all resulting queries to a pandas.DataFrame by wrapping the `pymongo` query in

```
df = pandas.DataFrame(list(db.imdb_movies_genres.find())
print(df)
	_id 	index 	director_id 	first_name 	last_name
0 	62da05f8e5d6d03453887957 	70115 	71645 	Martin 	Scorsese
```

### Second Helpful Tip: See the MongoDB SQL to Mongo mapping information to help: https://www.mongodb.com/docs/manual/reference/sql-comparison/

In [4]:
pandas.DataFrame(list(db.imdb_directors.find({"last_name" : "Scorsese"})))

,_id,index,director_id,first_name,last_name
0,62dab88b118486e1c4c718ae,70115,71645,Martin,Scorsese
1,62dabd6d19aa76b5d3914b6d,70115,71645,Martin,Scorsese


**Problem 1a**

Using pymongo, SELECT 10 movies from the imbd_movies table. Select 10 directors from imbd_directors and order by `first_name`.

In [5]:
pandas.DataFrame(list(db.imdb_movies.find().limit(10)))

,_id,index,movie_id,name,year,rank
0,62dab888118486e1c4c09b81,0,0,#28,2002,0.0
1,62dab888118486e1c4c09b82,1,1,"#7 Train: An Immigrant Journey, The",2000,0.0
2,62dab888118486e1c4c09b83,2,2,$,1971,6.4
3,62dab888118486e1c4c09b84,3,3,"$1,000 Reward",1913,0.0
4,62dab888118486e1c4c09b85,4,4,"$1,000 Reward",1915,0.0
5,62dab888118486e1c4c09b86,5,5,"$1,000 Reward",1923,0.0
6,62dab888118486e1c4c09b87,6,6,"$1,000,000 Duck",1971,5.0
7,62dab888118486e1c4c09b88,7,7,"$1,000,000 Reward, The",1920,0.0
8,62dab888118486e1c4c09b89,8,8,"$10,000 Under a Pillow",1921,0.0
9,62dab888118486e1c4c09b8a,9,9,"$100,000.00",1915,0.0


In [6]:
pandas.DataFrame(list(db.imdb_directors.find().sort('first_name',1).limit(10)))

,_id,index,director_id,first_name,last_name
0,62dab88b118486e1c4c616c2,4087,4175,A.,Balakrishnan
1,62dab88b118486e1c4c620cb,6656,6779,A.,Berry
2,62dab88b118486e1c4c6258b,7872,8026,A.,Bobrov
3,62dab88b118486e1c4c62223,7000,7125,A.,Bhimsingh
4,62dab88b118486e1c4c61973,4776,4871,A.,Barr-Smith
5,62dab88b118486e1c4c6238a,7359,7494,A.,Bistritsky
6,62dab88b118486e1c4c63a65,13210,13475,A.,Chandrasekaran
7,62dab88b118486e1c4c639ee,13091,13355,A.,Champeaux
8,62dab88b118486e1c4c61509,3646,3728,A.,Babes
9,62dab88b118486e1c4c60b0d,1090,1114,A.,Aleksandrov


**Problem 1b**

Using pymongo, how many movies are there? How many directors are there? 

In [32]:
pandas.DataFrame(list(db.imdb_directors.distinct("director_id")))

,0
0,1
1,2
2,3
3,4
4,5
...,...
86875,88797
86876,88798
86877,88799
86878,88800


In [33]:
pandas.DataFrame(list(db.imdb_movies.distinct("movie_id")))

,0
0,0
1,1
2,2
3,3
4,4
...,...
355141,378611
355142,378612
355143,378613
355144,378614


*Write your answer here*

**Problem 1c**

Using pymongo, determine how many movies are there after the year 2000?

In [34]:
pandas.DataFrame(list(db.imdb_movies.find({"year": {"$gt":2000}}).distinct("movie_id")
                     ))

,0
0,0
1,10
2,17
3,21
4,33
...,...
39581,378565
39582,378576
39583,378583
39584,378599


*Write your answer here*

**Problem 1d**

How many different movie genres are there?

In [35]:
pandas.DataFrame(list(db.imdb_movies_genres.distinct("genre")
                     ))

,0
0,Action
1,Adult
2,Adventure
3,Animation
4,Comedy
5,Crime
6,Documentary
7,Drama
8,Family
9,Fantasy


## Problem 2) Groups and Aggregates

Now that we know why the data has been organized in this way, we can leverage this unique structure in order to learn interesting properties of the data. 

**Problem 2a**

In which year were the most movies made according to IMDb?

In [ ]:
db.content.aggregate({"$group": {"$gt":2000}}).distinct("movie_id")
                     ))
db.contest.aggregate([
    {"$group" : {_id:"$province", count:{$sum:1}}}
])

*write your answer here*

**Problem 2b**

How many "Action" movies where made after the year 1980? Before the year 1980?

*write your answer here*

**Problem 3c**

Select all films made by `Scorsese`. How many are there?

*write your answer here*

**Problem 3c**

According the the IMDb data, which director has directed the most movies?

*write your answer here*

**Problem 3d**

According the the IMDb data, which director has directed the most movies in each genre?

*write your answer here*

## Challenge Problem) Make your own tables

**Problem 1a**

Create a new "collection".

**Problem 1b**

INSERT 3 "documentions" into the "collection you made above

**Problem 1c**

Create a pandas DataFrame and save the rows as "documents" in a new "collection" you make

*** hint look at using `pandas.to_dict` ****